La empresa está interesada en hacer crecer el canal de venta online, para eso es necesario entender cuáles son los factores determinantes para que una venta se realice por medio de ese canal. Se sugiere utilizar *Random Forest* como modelo para clasificación de ventas online.

Se dispoinibilizan los siguientes datasets:
* Venta.csv
* Productos.csv
* Sucursales.csv
* Clientes.csv

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

In [2]:
from IPython.display import clear_output

Se utiliza el archivo de ventas procesado:

In [8]:
df_ventas = pd.read_csv("../Datasets/Venta.csv", sep=",", parse_dates=["Fecha"])

In [9]:
df_ventas.head()

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3.0


In [10]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46645 entries, 0 to 46644
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   IdVenta        46645 non-null  int64         
 1   Fecha          46645 non-null  datetime64[ns]
 2   Fecha_Entrega  46645 non-null  object        
 3   IdCanal        46645 non-null  int64         
 4   IdCliente      46645 non-null  int64         
 5   IdSucursal     46645 non-null  int64         
 6   IdEmpleado     46645 non-null  int64         
 7   IdProducto     46645 non-null  int64         
 8   Precio         45721 non-null  float64       
 9   Cantidad       45753 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(6), object(1)
memory usage: 3.6+ MB


In [11]:
df_ventas.shape

(46645, 10)

In [12]:
import datetime as dt
df_ventas['Fecha']=df_ventas['Fecha'].map(dt.datetime.toordinal)
df_ventas.head(3)

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,736762,2018-03-17,3,969,13,1674,42817,813.12,2.0
1,2,737056,2018-12-29,2,884,13,1674,42795,543.18,3.0
2,3,736051,2016-03-31,2,1722,13,1674,42837,430.32,1.0


Se lee el maestro de Clientes para obtener la latitud, longitud y edad

In [15]:
df_clientes = pd.read_csv("../Datasets/Clientes.csv", sep=";")

Se lle el maestro de Sucursales para obtener latitud y longitud

In [17]:
df_sucursales = pd.read_csv("../Datasets/Sucursales.csv", sep=";")

Se puede utilizar la biblioteca Geopy para obtener la distancia entre el cliente y la sucursal:
    !pip install geopy

In [21]:
!pip install geopy

In [22]:
from geopy.distance import geodesic

In [23]:
punto1 = (-34.6157437, -58.573385) #lat-long de buenos aires
punto2 = (41.909986, 12.3959124) #lat-long de roma

In [24]:
type(punto1)

tuple

In [25]:
print(geodesic(punto1, punto2).km)

11139.019659518188


In [26]:
print(geodesic((-34.6157437, -58.573385), (41.909986, 12.3959124)).km)

11139.019659518188


Se trabaja sobre los dataframes maestros de clientes y sucursales para adecuar nombres de campos y tipos de datos, tambien para hacer tratamiento sobre valores faltantes

Como vemos que hay problemas en los datos, es decir, valores de latitud y longitud que no corresponden a Argentina, se procede a hacer correcciones, las cuáles serán de dos tipos:
* Casos donde la latitud y longitud están en positivo
* Casos donde la latitud y longitud están intercambiadas

Se agrega en el dataframe de ventas los datos de geoposición y edad

Los valores nulos pueden ser descartados o bien imputados con otro valor

Se genera el campo donde va a estar la distancia del cliente con la sucursal, que es la geodésica entre los dos puntos de coordenadas

El Canal de venta Online es el IdCanal = 2, se observan los datos con ese filtro

Del maestro de Productos, se obtiene el tipo de producto

Se analiza la correlación entre las variables

In [ ]:
def plot_corre_heatmap(corr):
    '''
    Definimos una función para ayudarnos a graficar un heatmap de correlación
    '''
    plt.figure(figsize=(12,10))
    sns.heatmap(corr, cbar = True,  square = False, annot=True, fmt= '.2f'
                ,annot_kws={'size': 15},cmap= 'coolwarm')
    plt.xticks(rotation = 45)
    plt.yticks(rotation = 45)
    # Arreglamos un pequeño problema de visualización
    b, t = plt.ylim() # discover the values for bottom and top
    b += 0.5 # Add 0.5 to the bottom
    t -= 0.5 # Subtract 0.5 from the top
    plt.ylim(b, t) # update the ylim(bottom, top) values
    plt.show()

In [ ]:
corr = df_ventas[['Precio','Cantidad','Fecha','Edad','Dist_Cli_Suc','IdTipoProducto','VentaOnline']].corr()
plot_corre_heatmap(corr)

Se analiza la distribución entre ventas online y el resto

Se generan los datos con los que se va a entrenar Random Forest

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import learning_curve

Se evaluan distintos hiperparámetros, analizando la curva de validación y de aprendizaje

In [ ]:
train_accuracy = []
test_accuracy = []
oob_scores = []

N_estimadores = [2,50,100,150,200,250,300]
i = 0
i_max = len(N_estimadores)
for estimadores in N_estimadores:
    i = i + 1
    clf = RandomForestClassifier(n_estimators=estimadores, n_jobs=-1, oob_score= True, random_state = 42)
    clf.fit(X_train,y_train)
    
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    train_accuracy.append(metrics.accuracy_score(y_train, y_train_pred))
    test_accuracy.append(metrics.accuracy_score(y_test, y_test_pred))
    oob_scores.append(clf.oob_score_)
    
    clear_output(wait=True)
    print('Completado: ' + str(round(i / i_max * 100, 2)) + '%')
    
train_accuracy = np.array(train_accuracy)
test_accuracy = np.array(test_accuracy)
oob_scores = np.array(oob_scores)

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(N_estimadores, train_accuracy, label = 'Train')
plt.plot(N_estimadores, test_accuracy, label = 'Test')
plt.plot(N_estimadores, oob_scores, label = 'OOB')
plt.xlabel('Numero de estimadores')
plt.ylabel('Accuracy')
plt.legend()
# plt.xlim(0,50)
plt.show()